In [1]:
import numpy as np
import twitter, re, datetime, pandas as pd

twitter_keys = {
    'consumer_key':        'L4sziHBqV4VUIfKezbos0JMVl',
    'consumer_secret':     'lJau6R7GIHFwoGR5wB3PlLQPXBChwzJFJ9WGXXtazcDSA1Vb1X',
    'access_token_key':    '941359629606539264-05XcmQfdwMXTbPNWS3r7cZThvbQBxCK',
    'access_token_secret': 'VdE3VJVk6oxbohQGcw7WYA5Tg4Sr8kW9duTO1wxmB6qXk'
}

api = twitter.Api(
    consumer_key         =   twitter_keys['consumer_key'],
    consumer_secret      =   twitter_keys['consumer_secret'],
    access_token_key     =   twitter_keys['access_token_key'],
    access_token_secret  =   twitter_keys['access_token_secret'],
    tweet_mode = 'extended'
)

In [15]:
x = api.GetUserTimeline(screen_name="realDonaldTrump", count=500, include_rts=False)
x = [_.AsDict() for _ in x]

In [16]:
x[0]

{'created_at': 'Wed Jan 29 19:41:50 +0000 2020',
 'favorite_count': 19185,
 'full_text': 'https://t.co/aX5WIX8B0O',
 'hashtags': [],
 'id': 1222605791074693122,
 'id_str': '1222605791074693122',
 'lang': 'und',
 'media': [{'display_url': 'pic.twitter.com/aX5WIX8B0O',
   'expanded_url': 'https://twitter.com/realDonaldTrump/status/1222605791074693122/video/1',
   'id': 1222602852532658176,
   'media_url': 'http://pbs.twimg.com/media/EPeRcNHWAAAPjTU.jpg',
   'media_url_https': 'https://pbs.twimg.com/media/EPeRcNHWAAAPjTU.jpg',
   'sizes': {'medium': {'w': 480, 'h': 270, 'resize': 'fit'},
    'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
    'small': {'w': 480, 'h': 270, 'resize': 'fit'},
    'large': {'w': 480, 'h': 270, 'resize': 'fit'}},
   'type': 'video',
   'url': 'https://t.co/aX5WIX8B0O',
   'video_info': {'aspect_ratio': [16, 9],
    'duration_millis': 360358,
    'variants': [{'content_type': 'application/x-mpegURL',
      'url': 'https://video.twimg.com/amplify_video/12226028

In [9]:
for element in x : 
    print (element['id'])
    print (element['full_text'])
    print ('---------------')

1222605791074693122
https://t.co/aX5WIX8B0O
---------------
1222601464889200653
For the first time in American History, we have replaced a disastrous Trade Deal that REWARDED outsourcing with a truly FAIR and RECIPROCAL Trade Deal that will keep jobs, wealth and growth right here in AMERICA! https://t.co/bDe8IV9DyD
---------------
1222600757389725698
Thanks to our pro-worker, pro-American economic policies, unemployment is at the lowest level in more than 50 years. We have created over 7 MILLION new JOBS. More Americans are working today than EVER before. We have the hottest ECONOMY on earth! https://t.co/TRGKRF7xgQ
---------------
1222589891785175041
USMCA is a cutting edge state of the art agreement that protects, defends and serves the great people of our Country. Promises Made, Promises Kept! https://t.co/PuiG1JPTTI
---------------
1222589334739673088
USMCA is a massive win for American manufacturers and auto workers! https://t.co/eOgFm31cAo
---------------
1222547304491646976
They

In [21]:
y = api.GetUserTimeline(screen_name="realDonaldTrump", count=20, max_id=1222605791074693122, include_rts=False)
y = [_.AsDict() for _ in y]

In [22]:
for element in y:
    print (element['id'])
    print(element['full_text'])
    print('--')

1222605791074693122
https://t.co/aX5WIX8B0O
--
1222601464889200653
For the first time in American History, we have replaced a disastrous Trade Deal that REWARDED outsourcing with a truly FAIR and RECIPROCAL Trade Deal that will keep jobs, wealth and growth right here in AMERICA! https://t.co/bDe8IV9DyD
--
1222600757389725698
Thanks to our pro-worker, pro-American economic policies, unemployment is at the lowest level in more than 50 years. We have created over 7 MILLION new JOBS. More Americans are working today than EVER before. We have the hottest ECONOMY on earth! https://t.co/TRGKRF7xgQ
--
1222589891785175041
USMCA is a cutting edge state of the art agreement that protects, defends and serves the great people of our Country. Promises Made, Promises Kept! https://t.co/PuiG1JPTTI
--
1222589334739673088
USMCA is a massive win for American manufacturers and auto workers! https://t.co/eOgFm31cAo
--
1222547304491646976
They will never be satisfied! https://t.co/MXHFQXWC80
--
122254681309

In [23]:
y[0]

dict

In [24]:
#TweetMiner function from Mike Roman

class TweetMiner(object):

    
    def __init__(self, api, result_limit = 20):
        
        self.api = api        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="HillaryClinton", mine_retweets=False, max_pages=20):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   item['retweet_count'],
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                except:
                        mined = {
                        'tweet_id':        item['id'],
                        'handle':          item['user']['screen_name'],
                        'retweet_count':   0,
                        'text':            item['full_text'],
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item['created_at'],
                    }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [50]:
twitter_miner = TweetMiner(api, result_limit=500)

In [51]:
hilary = twitter_miner.mine_user_tweets(user="HillaryClinton")
donald = twitter_miner.mine_user_tweets(user="realDonaldTrump")

In [52]:
for x in range(10):
    print (hilary[x]['text'])
    print('---------------------')

@FullFrontalSamB I really do.
---------------------
Love this. 

@StephenAtHome, @sethmeyers, @jimmykimmel, @jimmyfallon, @JKCorden, @TrevorNoah—I hope you'll take up @IAmSamBee on this challenge! https://t.co/OakC0K9K0Y
---------------------
Congratulations to @AshantiGholar and @EmergeAmerica! Excited to work with you to recruit and help elect a historic number of women to office. https://t.co/PImX9RiW84
---------------------
Keep calling your Republican senators: 

(202) 224-3121 

#WeWantWitnesses https://t.co/mfW13LKb6t
---------------------
Leaving Sundance having seen some terrific films, including “Siempre, Luis.” It’s the story of how @Vegalteno has committed his life to activism, progressive values, &amp; Puerto Rico, all while raising a remarkable family (👋@Lin_Manuel). Check it out when you can!
---------------------
A trial that doesn't allow testimony from material witnesses isn't a trial at all. It's a cover-up.

Your Republican senators need to hear from you today. (202

In [53]:
for x in range(10):
    print (donald[x]['text'])
    print('------------------')

GAME OVER! https://t.co/yvMa6bPqfy
------------------
https://t.co/aX5WIX8B0O
------------------
For the first time in American History, we have replaced a disastrous Trade Deal that REWARDED outsourcing with a truly FAIR and RECIPROCAL Trade Deal that will keep jobs, wealth and growth right here in AMERICA! https://t.co/bDe8IV9DyD
------------------
Thanks to our pro-worker, pro-American economic policies, unemployment is at the lowest level in more than 50 years. We have created over 7 MILLION new JOBS. More Americans are working today than EVER before. We have the hottest ECONOMY on earth! https://t.co/TRGKRF7xgQ
------------------
USMCA is a cutting edge state of the art agreement that protects, defends and serves the great people of our Country. Promises Made, Promises Kept! https://t.co/PuiG1JPTTI
------------------
USMCA is a massive win for American manufacturers and auto workers! https://t.co/eOgFm31cAo
------------------
They will never be satisfied! https://t.co/MXHFQXWC80
-

In [64]:
#Hilary DF 
pd.DataFrame(hilary)

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jan 29 21:04:16 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,97,@FullFrontalSamB I really do.,1222626538396569600
1,Wed Jan 29 19:18:36 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,1193,"Love this. \n\n@StephenAtHome, @sethmeyers, @j...",1222599945531875330
2,Wed Jan 29 17:51:42 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,374,Congratulations to @AshantiGholar and @EmergeA...,1222578076548288512
3,Wed Jan 29 17:01:30 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,9644,Keep calling your Republican senators: \n\n(20...,1222565443401146369
4,Tue Jan 28 20:08:57 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,1031,Leaving Sundance having seen some terrific fil...,1222250229518434306
5,Mon Jan 27 18:00:31 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,19290,A trial that doesn't allow testimony from mate...,1221855517750059008
6,Fri Jan 24 15:49:49 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,12086,Every American should watch this.\n\n#RightMat...,1220735463746609159
7,Thu Jan 23 20:01:24 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,1107,"Jim Lehrer was a steady voice for truth, a pro...",1220436388383989761
8,Thu Jan 23 17:08:05 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,7213,Fact check:\n\nTrue. https://t.co/XkO8Hldhbc,1220392771737079808
9,Thu Jan 23 16:02:28 +0000 2020,HillaryClinton,2020-01-29 18:02:00.247156,20766,The entire world is waiting to see whether Sen...,1220376259374481409


In [58]:
miner = TweetMiner(api, result_limit=2500)
trump_tweets = miner.mine_user_tweets("realDonaldTrump")

In [59]:
trump_df = pd.DataFrame(trump_tweets)

In [60]:
trump_df

,created_at,handle,mined_at,retweet_count,text,tweet_id
0,Wed Jan 29 23:00:42 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,1217,GAME OVER! https://t.co/yvMa6bPqfy,1222655839573565440
1,Wed Jan 29 19:41:50 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,6349,https://t.co/aX5WIX8B0O,1222605791074693122
2,Wed Jan 29 19:24:38 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,9221,"For the first time in American History, we hav...",1222601464889200653
3,Wed Jan 29 19:21:50 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,9163,"Thanks to our pro-worker, pro-American economi...",1222600757389725698
4,Wed Jan 29 18:38:39 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,8599,USMCA is a cutting edge state of the art agree...,1222589891785175041
5,Wed Jan 29 18:36:26 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,9666,USMCA is a massive win for American manufactur...,1222589334739673088
6,Wed Jan 29 15:49:25 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,12118,They will never be satisfied! https://t.co/MXH...,1222547304491646976
7,Wed Jan 29 15:47:28 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,10419,“The best evidence is the Transcript itself.” ...,1222546813099659264
8,Wed Jan 29 15:39:04 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,13276,There is much talk that certain Democrats are ...,1222544696964263936
9,Wed Jan 29 15:33:00 +0000 2020,realDonaldTrump,2020-01-29 18:02:35.454389,15321,On the Iraq War Resolution being voted on tomo...,1222543172192129024


In [65]:
trump_df.shape

(1087, 6)

In [67]:
hilary_tweets = twitter_miner.mine_user_tweets('HillaryClinton')

In [68]:
hilary_df = pd.DataFrame(hilary_tweets)

In [70]:
print (hilary_df.shape)

(2554, 6)


In [72]:
tweets = pd.concat([trump_df, hilary_df], axis=0)
tweets.shape

(3641, 6)

In [74]:

from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

# Pulls all of trumps tweet text's into one giant string
summaries = "".join(trump_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)


[('impeachment hoax', 44),
 ('united states', 35),
 ('republican party', 32),
 ('witch hunt', 30),
 ('radical left', 30),
 ('fake news', 29),
 ('nancy pelosi', 29),
 ('left democrats', 21),
 ('president trump', 21),
 ('radical left democrats', 20),
 ('american people', 17),
 ('read transcripts', 17),
 ('white house', 16),
 ('stock market', 16),
 ('trade deal', 15),
 ('great job', 15),
 ('shifty schiff', 15),
 ('new york', 13),
 ('approval rating', 12),
 ('corrupt politician', 12)]

In [76]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')

summaries = "".join(hilary_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)

Counter(ngrams_summaries).most_common(20)

[('donald trump', 112),
 ('https ttgeqxnqym', 79),
 ('make sure', 65),
 ('hillary https', 65),
 ('vote https', 62),
 ('health care', 48),
 ('debatenight https', 47),
 ('ttgeqxnqym https', 45),
 ('https ttgeqxnqym https', 45),
 ('https 3tkj4h68kz', 44),
 ('human rights', 37),
 ('hillary clinton', 34),
 ('3tkj4h68kz https', 32),
 ('https 3tkj4h68kz https', 32),
 ('potus https', 29),
 ('climate change', 28),
 ('today https', 28),
 ('trump https', 26),
 ('president https', 25),
 ('debate https', 25)]